# Вебинар 8. Консультация

---

## 1. Recap по финальному проекту

**Основное**
- Дедлайн - 17 января 23:59
- Целевая метрика precision@5
- Бейзлайн решения - [MainRecommender](https://github.com/geangohn/recsys-tutorial/blob/master/src/recommenders.py)
- Сдаем ссылку на github с решением. На github должен быть файл recommendations.csv (user_id | [rec_1, rec_2, ...] с рекомендациями. rec_i - реальные id item-ов (из retail_train.csv)
- Минимальный скор на Тесте 0.18 (retail_test.csv)

**Hints:** 

Сначала просто попробуйте разные параметры MainRecommender:  
- N в топ-N товарах при формировании user-item матирцы (сейчас топ-5000)  
- Различные веса в user-item матрице (0/1, кол-во покупок, log(кол-во покупок + 1), сумма покупки, ...)  
- Разные взвешивания матрицы (TF-IDF, BM25 - у него есть параметры)  
- Разные смешивания рекомендаций (обратите внимание на бейзлайн - прошлые покупки юзера)  

Сделайте MVP - минимально рабочий продукт - (пусть даже top-popular), а потом его улучшайте

Если вы делаете двухуровневую модель - следите за валидацией 

**Альтернатива**
https://www.kaggle.com/competitions/h-and-m-personalized-fashion-recommendations
если попадете в топ 30% приватного лидерборда - зачту

---

## 2. Ответы на вопросы

Как делать кросс-валидацию?
- [Статья](https://www.ethanrosenthal.com/2016/10/19/implicit-mf-part-1/)
- Смотреть на метрику на отложенной выборке, меняя гиперпараметры

Какие гиперпараметры оптимизировать у ALS:
- Кол-во факторов 
- regularization / iterations

Как можно делать кросс-валидацию (отложенная выборка)

In [ ]:
from src.recommenders import MainRecommender

my_model = MainRecommender(data)
user_item_matrix = my_model.user_item_matrix

for factors in [20, 50, 100]:
    my_model.fit(user_item_matrix, n_factors=factors, regularization=0.001, iterations=15, num_threads=4)
    # calculate precision@5 on validation

In [ ]:
# your_questions

---

## 3. Разбор соревнования по рекомендациям топ-30 товаров X5 Retail Hero

Постановка задачи + особенности данных + подход к решению из [презентации](https://vk.com/away.php?utf=1&to=https%3A%2F%2Fgithub.com%2Faprotopopov%2Fretailhero_recommender%2Fblob%2Fmaster%2Fslides%2Fretailhero_recommender.pdf) решения 1-ого места

Бонус - [код](https://github.com/aprotopopov/retailhero_recommender) решения 1-ого места

## Основные подходы участников из топ-15

### Генерация кандидатов

В основном генерировали k = {50, 100} кандидатов, чтобы модель 2-ого уровня отрабатывала за разумное время

- Прошлые покупки юзера + топ-популярных, если покупок < k --> recall@100 ~ 42%
- Из ALS, item2item моделей
- implicit.nn.CosineRecommender

### Фичи в модели 2-ого уровня
Коллаборативные:
- biases + embeddings из коллаборативной фильтрации / Скоры item2item моделей
- TF-IDF матирца item-ов c > N покупками
- TF-IDF + TSNE/UMAP

handcrafted фичи для товаров:
- категории товаров
- нормированная частота покупки товара для каждого клиента
- кол-во магазинов, в которых продавался товар
- кол-во транзакций клиента
- mean / max / std кол-ва уникальных товаров в корзине клиента 
- mean / max / std кол-ва уникальных категорий в корзине клиента 

handcrafted фичи для юзеров:
- Средний чек
- Средняя цена одного купленного товара
- Среднее кол-во дней между покупками / с последней покупки    
- Количество уникальных покупок по всем категориям transaction_id, product_id, store_id, level_i_id
- Признаки с накопленными бонусами
- Средняя скидка, доля купленных товаров со скидками

Интересные:
- Факт заказа каждого товара в последних 5 транзакциях в виде последовательности бит (категориальная фича). 
10001 - купил товар в последней транзакции и 5 транзакий назад (feature hashing)
- item co_ocurrence
- word2vec эмбеддинги товаров (альтернативное название - item2vec, prod2vec)
- Расстояние от word2vec эмбеддинга товара до среднего эмбедиингов товаров, купленных юзером

In [ ]:
# Работа с эмбеддингами товаров на примере ALS вместо word2vec
from implicit.als import AlternatingLeastSquares

model = AlternatingLeastSquares()
model.fit(...)

model.item_factors  # Добавить как фичи в модель 2-ого уровня

# Предположение: эмбеддинг юзера = среднее эмбеддингов купленниых им товаров
# эмбеддинг товара - эмбеддинг юзера  # чем разница меньше, тем товар ближе к интересам пользователя

### Модели 2-ого уровня
- классификация через LightGBM
- ранжирование CatBoost через YetiRank, YetiRankPairwise

# Light auto ML

https://lightautoml.readthedocs.io/en/latest/

https://lightautoml.readthedocs.io/en/latest/generated/lightautoml.tasks.base.Task.html

In [1]:
!pip install lightautoml

     |████████████████████████████████| 296 kB 14.3 MB/s 
     |████████████████████████████████| 425 kB 74.4 MB/s 
     |████████████████████████████████| 1.2 MB 60.7 MB/s 
     |████████████████████████████████| 215 kB 76.3 MB/s 
     |████████████████████████████████| 76.6 MB 1.3 MB/s 
     |████████████████████████████████| 308 kB 59.5 MB/s 
     |████████████████████████████████| 9.5 MB 83.7 MB/s 
     |████████████████████████████████| 804.1 MB 3.0 kB/s 
     |████████████████████████████████| 2.8 MB 55.5 MB/s 
     |████████████████████████████████| 81 kB 7.7 MB/s 
     |████████████████████████████████| 210 kB 68.9 MB/s 
     |████████████████████████████████| 78 kB 7.4 MB/s 
     |████████████████████████████████| 49 kB 7.1 MB/s 
     |████████████████████████████████| 113 kB 70.5 MB/s 
     |████████████████████████████████| 146 kB 50.5 MB/s 
     |████████████████████████████████| 21.0 MB 1.2 MB/s 
     |████████████████████████████████| 23.2 MB 1.3 MB/s 
     |█████████████

In [5]:
from lightautoml.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML
from lightautoml.tasks import Task
from lightautoml.tasks.common_metric import mean_quantile_error, rmsle
import pandas as pd

In [3]:
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/My Drive/GB/RecSys/webinar_6/targets_lvl_2.csv'

Mounted at /content/drive


In [10]:
# также как lightgbm
train_data = pd.read_csv(path, sep=';', index_col=0)

In [11]:
train_data.head()

,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
0,2070,1105426,0.0,69,DELI,Private,SANDWICHES,SANDWICHES - (COLD),,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown
1,2070,1097350,0.0,2468,GROCERY,National,DOMESTIC WINE,VALUE GLASS WINE,4 LTR,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown
2,2070,879194,0.0,69,DRUG GM,Private,DIAPERS & DISPOSABLES,BABY DIAPERS,14 CT,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown
3,2070,948640,0.0,1213,DRUG GM,National,ORAL HYGIENE PRODUCTS,WHITENING SYSTEMS,3 OZ,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown
4,2070,928263,0.0,69,DRUG GM,Private,DIAPERS & DISPOSABLES,BABY DIAPERS,13 CT,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown


In [12]:
TASK = Task('reg', loss='mse', metric=mean_quantile_error, greater_is_better=False)
TIMEOUT = 300000
N_THREADS = 4
N_FOLDS = 5
RANDOM_STATE = 42
TARGET_NAME = 'target'
TEST_SIZE=0.2

In [13]:
roles = {'target': TARGET_NAME, 'drop': ['user_id', 'item_id']}

In [14]:
automl_model = TabularAutoML(task=TASK,
                            timeout=TIMEOUT,
                            cpu_limit = N_THREADS,
                            gpu_ids='all',
                            reader_params = {'n_jobs': N_THREADS, 'cv': N_FOLDS, 'random_state': RANDOM_STATE},
                             
                            general_params={'use_algos': [ ['lgb_tuned', 'cb_tuned', 'cb', 'lgb'], ['lgb_tuned', 'cb'] ]},
                             
                            tuning_params={'max_tuning_iter': 10},
                      )

In [ ]:
oof_pred = automl_model.fit_predict(train_data, roles = roles)